
CSYE7105 High Performance Parallel Machine Learning and AI

Instructor: Dr. Handan Liu

Example: Parallelize machine Learning with sklearn and Dask


In [1]:
from sklearn.utils import parallel_backend


### Scikit-Learn & Joblib


In [2]:
import numpy as np
from dask.distributed import Client

import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

In [3]:
digits = load_digits()

In [4]:
param_space = {
    'C': np.logspace(-6, 6, 13),
    'gamma': np.logspace(-8, 8, 17),
    'tol': np.logspace(-4, -1, 4),
    'class_weight': [None, 'balanced'],
}



### Experiment: 
    Using 3 parallel strategies on same 8 workers 


In [5]:
%%timeit
model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=4, n_iter=20, n_jobs=8) 
search.fit(digits.data, digits.target)

19.2 s ± 1.01 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
from joblib import parallel_backend

with parallel_backend('threading', n_jobs=4):         
    model = SVC(kernel='rbf')
    search = RandomizedSearchCV(model, param_space, cv=4, n_iter=20, n_jobs=2) 
    search.fit(digits.data, digits.target)

2.86 s ± 139 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
# 3. 
client = Client(n_workers=2, processes=False, threads_per_worker=4)        

In [7]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.99.249.84:8787/status,
Dashboard: http://10.99.249.84:8787/status,Workers: 2
Total threads: 8,Total memory: 20.00 GiB
Status: running,Using processes: False
Comm: inproc://10.99.249.84/61726/1,Workers: 2
Dashboard: http://10.99.249.84:8787/status,Total threads: 8
Started: Just now,Total memory: 20.00 GiB
Comm: inproc://10.99.249.84/61726/4,Total threads: 4
Dashboard: http://10.99.249.84:44209/status,Memory: 10.00 GiB
Nanny: None,


In [9]:
%%timeit
model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=4, n_iter=20)
with joblib.parallel_backend('dask'):
    search.fit(digits.data, digits.target)

2.95 s ± 138 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
